## Build big dataset 

In [1]:
import pandas as pd
Z = [0.0002, 0.0004, 0.0008, 0.0012, 0.0016, 0.002, 0.004, 0.006, 0.008, 0.012, 0.016, 0.02]
alpha = [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
f_MT= [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]


def write_data(data_col = [6], name_col = ["time_delay"],
             A = alpha[0],  A_df = False,
             fMT = f_MT[0], fMT_df =False,  
             Z = Z[1], Z_df=False):   
    
    ##ensure col section is always a list ready for python shortcuts
    if not isinstance(data_col, list):data_col = [data_col]
    if not isinstance(Z, list):Z = [Z]
    if not isinstance(A, list):A = [A]
    if not isinstance(fMT, list):fMT = [fMT]
    #fetch the associated col labels
    col_head = ['ID', 'm_iZMAS_1', "m_iZMAS_2", "m_comp_1","m_comp_2", "m_rem", "t_delay", "smj_axis", "orb_ecc"]
    name_col = [col_head[x] for x in data_col] 
    
    out_path = ("~/project_lcp/results/"+
                "fMT_"+str(f_MT[0]) +"to" + str(f_MT[-1]) +  
                "A_"+str(A[0]) +"to"+str(A[-1])+ 
                "Z_"+str(Z[0])+"to"+str(Z[-1])+
                "_BHBs.csv")
    start_df = True 
    for Z_ in Z:
        Z_lb = str(Z_)
        for A_ in A:
            A_lb = str(A_)        
            for fMT_ in fMT:
                fMT_lb=str(fMT_).replace('.', '')
                filepath = "~/project_lcp/data/vnew_delayed_fMT"+fMT_lb + "/A"+A_lb +"/data_BHBs_"+Z_lb+".txt"
                print("Z = ", Z_, ", A = ", A_, ", fMT =", fMT_)
                
                try:
                    df_out = pd.read_csv(filepath, sep=" ", skiprows=[0], header=None, usecols=data_col, names =name_col)
                    if A_df: df_out["alpha"]= A_
                    if fMT_df: df_out["f_MT"]= fMT_
                    if Z_df: df_out["Z"]= Z_
                    if start_df: #this way will start later if first is missing
                        df_out.to_csv(out_path, index=False)#create file
                        start_df = False
                    else:
                        df_out.to_csv(out_path, mode='a', header=False, index = False)
                except:
                    print(filepath+": Does not exist")
                
                
    print("complete")

In [ ]:
import time
t1 = time.time()
df = write_data(data_col=[8], fMT=f_MT, A=alpha, Z=Z, names =["orb_ecc"])
t2 = time.time()
dt = t2 -t1
print(dt)

### Using Parquet in Pandas

In [2]:
import pandas as pd
df = pd.read_parquet("all_BHBs.parquet")
df

,ID,m_ZAMS_1,m_ZAMS_2,m_comp_1,m_comp_2,m_merge,t_delay,smj_axis,orb_ecc,alpha,f_MT,Z
0,1,54.3396,52.0830,22.1118,20.4424,42.5542,0.008093,4.4282,0.011750,0.5,0.05,0.0002
1,5,58.9754,55.9686,24.3563,22.2701,46.6264,0.010175,5.4175,0.010724,0.5,0.05,0.0002
2,9,96.3555,84.2747,38.3529,34.0211,72.3740,0.272228,19.9710,0.045441,0.5,0.05,0.0002
3,10,61.2173,58.7969,25.2146,23.5961,48.8107,0.008453,5.1401,0.010244,0.5,0.05,0.0002
4,11,51.9740,48.2456,21.6566,18.5773,40.2339,0.044590,8.9334,0.352150,0.5,0.05,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...
87756765,3118210,50.6832,50.5820,3.9894,3.9378,7.9272,0.006562,61.7820,0.996770,5,0.5,0.02
87756766,3409500,142.2995,24.4715,9.9454,3.3375,13.2829,1.176829,146.5100,0.986530,5,0.5,0.02
87756767,3550378,54.8366,54.7393,4.4193,4.3622,8.7815,0.712588,181.3500,0.993970,5,0.5,0.02
87756768,3562656,36.7711,33.6648,4.2631,3.3264,7.5895,1.782024,6.1815,0.233690,5,0.5,0.02


In [3]:
df.dtypes

ID             int64
m_ZAMS_1     float64
m_ZAMS_2     float64
m_comp_1     float64
m_comp_2     float64
m_merge      float64
t_delay      float64
smj_axis     float64
orb_ecc      float64
alpha       category
f_MT        category
Z           category
dtype: object

In [4]:
medians = df.groupby(['alpha',"f_MT", "Z"])['t_delay'].median().to_frame()
medians

t_delay
alpha f_MT Z               
0.5   0.05 0.0002  0.049587
           0.0004  0.134741
           0.0008  0.204421
           0.0012  0.335788
           0.0016  0.494622
...                     ...
10    1    0.006   7.221333
           0.008   3.374717
           0.012   4.245677
           0.016   1.822305
           0.02         NaN

[2640 rows x 1 columns]

### Loading a  few columns

In [3]:
col_lb = ["ID",      "m_ZAMS_1", "m_ZAMS_2", "m_comp_1", "m_comp_2", "m_merge",
            "t_delay", "smj_axis", "orb_ecc",  "alpha",    "f_MT",     "Z"]

df = pd.read_parquet("all_BHBs.parquet", columns = col_lb[4:9]  )
df


,m_comp_2,m_merge,t_delay,smj_axis,orb_ecc
0,20.4424,42.5542,0.008093,4.4282,0.011750
1,22.2701,46.6264,0.010175,5.4175,0.010724
2,34.0211,72.3740,0.272228,19.9710,0.045441
3,23.5961,48.8107,0.008453,5.1401,0.010244
4,18.5773,40.2339,0.044590,8.9334,0.352150
...,...,...,...,...,...
87756765,3.9378,7.9272,0.006562,61.7820,0.996770
87756766,3.3375,13.2829,1.176829,146.5100,0.986530
87756767,4.3622,8.7815,0.712588,181.3500,0.993970
87756768,3.3264,7.5895,1.782024,6.1815,0.233690


## Timings

In [2]:
import time
import pandas as pd
df=0
t1 = time.time()
df = pd.read_parquet("all_BHBs.parquet")
t2 = time.time()
dt = t2 -t1
print("Load time = ", dt, "/s")

t1 = time.time()
medians = df.groupby(['alpha',"f_MT", "Z"])['t_delay'].median().to_frame()
t2 = time.time()
dt = t2 -t1

medians
print("Grouping time = ", dt, "/s")

Load time =  40.815919399261475 /s
Grouping time =  6.335631608963013 /s
